# Configure The Physical Device

In notebook, we configure a device as the IoT edge device. We use a NC6 Ubuntu Linux VM as the edge device, which is the same Linux VM where you run the current notebook. The goal is to configure the edge device so that it can run [Docker](https://docs.docker.com/v17.12/install/linux/docker-ee/ubuntu), [nvidia-docker](https://github.com/NVIDIA/nvidia-docker), and [IoT Edge runtime](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux). If another device is used as the edge device, instructions need to be adjusted accordingly. 


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
from dotenv import set_key, get_key, find_dotenv

In [ ]:
# get the .env file where all the variables are stored
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
# get previously stored variables
device_connection_string = get_key(env_path, 'device_connection_string')

### Install az-cli iot extension 

In [8]:
accounts = !az account list --all -o tsv
if "Please run \"az login\" to access your accounts." in accounts[0]:
    !az login -o table
else:
    print("Already logged in")

Already logged in


### Register Microsoft key and software repository feed

Prepare your device for the IoT Edge runtime installation.

In [19]:
# Install the repository configuration. Replace <release> with 16.04 or 18.04 as appropriate for your release of Ubuntu.
release = !lsb_release -r
release  = release[0].split('\t')[1]
print(release)

16.04


In [20]:
!curl https://packages.microsoft.com/config/ubuntu/$release/prod.list > ./microsoft-prod.list

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    729      0 --:--:-- --:--:-- --:--:--   731


In [21]:
# Copy the generated list.
!sudo cp ./microsoft-prod.list /etc/apt/sources.list.d/

In [22]:
#Install Microsoft GPG public key
!curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
!sudo cp ./microsoft.gpg /etc/apt/trusted.gpg.d/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   8871      0 --:--:-- --:--:-- --:--:--  8855


### Install the Azure IoT Edge Security Daemon

The IoT Edge security daemon provides and maintains security standards on the IoT Edge device. The daemon starts on every boot and bootstraps the device by starting the rest of the IoT Edge runtime.
The installation command also installs the standard version of the iothsmlib if not already present.

In [23]:
# Perform apt update.
!sudo apt-get update

Get:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Ign:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Get:2 file:/var/nv-tensorrt-repo-ga-cuda9.0-trt3.0.2-20180108  InRelease
Ign:2 file:/var/nv-tensorrt-repo-ga-cuda9.0-trt3.0.2-20180108  InRelease
Get:3 file:/var/nvidia-diag-driver-local-repo-410.79  InRelease
Ign:3 file:/var/nvidia-diag-driver-local-repo-410.79  InRelease
Get:4 file:/var/nccl-repo-2.2.13-ga-cuda9.2  Release [574 B]
Hit:5 http://azure.archive.ubuntu.com/ubuntu xenial InRelease
Get:6 http://azure.archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:4 file:/var/nccl-repo-2.2.13-ga-cuda9.2  Release [574 B]                   
Get:7 http://azure.archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:8 file:/var/nv-tensorrt-repo-ga-cuda9.0-trt3.0.2-20180108  Release [574 B] 
Get:8 file:/var/nv-tensorrt-repo-ga-cuda9.0-trt3.0.2-20180108  Release [574 B] 
Get:9 file:/var/nvidia-diag-driver-local-repo-410.79  Release [574 B]          
Get:9 file:/v

In [24]:
# Install the security daemon. The package is installed at /etc/iotedge/.
!sudo apt-get install iotedge -y --no-install-recommends

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libiothsm-std
The following NEW packages will be installed:
  iotedge libiothsm-std
0 upgraded, 2 newly installed, 0 to remove and 130 not upgraded.
Need to get 5,510 kB of archives.
After this operation, 26.9 MB of additional disk space will be used.
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 libiothsm-std amd64 1.0.8-2 [422 kB]
Get:2 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 iotedge amd64 1.0.8-2 [5,088 kB]
Fetched 5,510 kB in 0s (26.2 MB/s) 
Selecting previously unselected package libiothsm-std.
(Reading database ... 432683 files and directories currently installed.)
Preparing to unpack .../libiothsm-std_1.0.8-2_amd64.deb ...
Unpacking libiothsm-std (1.0.8-2) ...
Selecting previously unselected package iotedge.
Preparing to unpack .../iotedge_1.0.8-2_amd64.deb ...
Adding system us

### Configure the Azure IoT Edge Security 

Configure the IoT Edge runtime to link your physical device with a device identity that exists in an Azure IoT hub.
The daemon can be configured using the configuration file at /etc/iotedge/config.yaml. The file is write-protected by default, you might need elevated permissions to edit it.

In [153]:
# Manual provisioning IoT edge device
!sudo sed -i "s#\(device_connection_string: \).*#\1\"$device_connection_string\"#g" /etc/iotedge/config.yaml

In [160]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(10) # Wait 10 seconds for iotedge to restart

In [162]:
# restart the daemon again
!sudo systemctl restart iotedge

### Verify successful installation

In [28]:
# check the status of the IoT Edge Daemon
!systemctl status iotedge

● iotedge.service - Azure IoT Edge daemon
   Loaded: loaded (/lib/systemd/system/iotedge.service; enabled; vendor preset: enabled)
   Active: active (running) since Wed 2019-07-31 20:34:24 UTC; 10s ago
     Docs: man:iotedged(8)
 Main PID: 67831 (iotedged)
    Tasks: 13
   Memory: 5.1M
      CPU: 2.739s
   CGroup: /system.slice/iotedge.service
           └─67831 /usr/bin/iotedged -c /etc/iotedge/config.yaml

Jul 31 20:34:27 gpuvm iotedged[67831]: 2019-07-31T20:34:27Z [INFO] - Listen...I.
Jul 31 20:34:27 gpuvm iotedged[67831]: 2019-07-31T20:34:27Z [INFO] - Checki...us
Jul 31 20:34:27 gpuvm iotedged[67831]: 2019-07-31T20:34:27Z [INFO] - Creati...nt
Jul 31 20:34:27 gpuvm iotedged[67831]: 2019-07-31T20:34:27Z [INFO] - Updati...nt
Jul 31 20:34:27 gpuvm iotedged[67831]: 2019-07-31T20:34:27Z [INFO] - Pullin.....
Jul 31 20:34:31 gpuvm iotedged[67831]: 2019-07-31T20:34:31Z [INFO] - Succes....0
Jul 31 20:34:31 gpuvm iotedged[67831]: 2019-07-31T20:34:31Z [INFO] - Creati.....
Jul 31 20:34:33 gpuvm

In [29]:
# Examine daemon logs
!journalctl -u iotedge --no-pager --no-full

-- Logs begin at Wed 2019-07-31 12:51:08 UTC, end at Wed 2019-07-31 20:34:35 UTC. --
Jul 31 20:34:01 gpuvm systemd[1]: Started Azure IoT Edge daemon.
Jul 31 20:34:01 gpuvm iotedged[67551]: 2019-07-31T20:34:01Z [INFO] - Using ...ml
Jul 31 20:34:01 gpuvm iotedged[67551]: 2019-07-31T20:34:01Z [INFO] - Starti...on
Jul 31 20:34:01 gpuvm iotedged[67551]: 2019-07-31T20:34:01Z [INFO] - Versio...0)
Jul 31 20:34:01 gpuvm systemd[1]: iotedge.service: Main process exited, cod...RE
Jul 31 20:34:01 gpuvm systemd[1]: iotedge.service: Unit entered failed state.
Jul 31 20:34:01 gpuvm systemd[1]: iotedge.service: Failed with result 'exit...'.
Jul 31 20:34:01 gpuvm systemd[1]: iotedge.service: Service hold-off time ov...t.
Jul 31 20:34:01 gpuvm systemd[1]: Stopped Azure IoT Edge daemon.
Jul 31 20:34:01 gpuvm systemd[1]: Started Azure IoT Edge daemon.
Jul 31 20:34:01 gpuvm iotedged[67581]: 2019-07-31T20:34:01Z [INFO] - Using ...ml
Jul 31 20:34:01 gpuvm iotedged[67581]: 2019-07-31T20:34:01Z [INFO] - Starti

When you run `docker ps` command in the edge device, you should see `edgeAgent` container is up running.

In [156]:
!docker ps

CONTAINER ID        IMAGE                                      COMMAND                   CREATED              STATUS              PORTS               NAMES
348f4339ca47        mcr.microsoft.com/azureiotedge-agent:1.0   "/bin/sh -c 'echo \"$…"   About a minute ago   Up 11 seconds                           edgeAgent


Next we will proceed with notebook [03_BuildRegisterImage.ipynb](03_BuildRegisterImage.ipynb).